(2a) First read in the sms-spam-bow.csv. I'm really not sure how to utilize it yet though...

In [6]:
bow_representation_of_messages = np.loadtxt("data/sms-spam-bow.csv", delimiter=" ", dtype=int)
print(bow_representation_of_messages.shape)


(5574, 8443)


Now we have the x_j^t BOW matrix, where j is the word-index column-wise, ranging from 0 to 8441 (first column is the ham(0)/spam(1) flag), and t is the sample index ranging from 0 to 5573.
First, let's find how many ham/spam messages there are

In [12]:
N = bow_representation_of_messages.shape[0]

# Select 80% for training purposes, the rest for testing
N_train = int(N*0.8)
N_test = N - N_train

training_set = bow_representation_of_messages[:N_train, :]
test_set = bow_representation_of_messages[N_train:, :]


Now, find all the parameters necessary for the classifier based on the training data

In [81]:

which_class = training_set[:, 0]
rows_with_ham=np.where(which_class==0)
rows_with_spam=np.where(which_class==1)


N_ham = np.size(rows_with_ham)
N_spam = np.size(rows_with_spam)

print ("Spams: ", N_spam, "Hams: ", N_ham)

BOW_samples = bow_representation_of_messages[:, 1:]
hams = BOW_samples[rows_with_ham,:]
spams = BOW_samples[rows_with_spam,:]

# Now, aggregate this matrix vertically. That is, find number of occurances of word j in all spam and ham "documents".
N_hams_j = np.sum(hams, axis=1)
N_spams_j = np.sum(spams, axis=1)

# These are used for estimators for p_ij (eq. 5.31)
rel_freq_hams = N_hams_j/N_ham
rel_freq_spams = N_spams_j/N_spam
log_p0j = np.log(rel_freq_hams)
log_p1j = np.log(rel_freq_spams)
log_p0j[np.isneginf(log_p0j)] = -100000000000 # Some probabilities are 0. They turn to -Inf when taking the log.
log_p1j[np.isneginf(log_p1j)] = -100000000000 # We need to set a very large negative value here to compensate for this

log_p0j_inv = np.log(1-rel_freq_hams)
log_p1j_inv = np.log(1-rel_freq_spams)


# The priors
P_ham = N_ham/N_train
P_spam = N_spam/N_train

# Now we are ready to write the discriminant functions for ham/spam. The input x is the bow representation of a message
# Note: The dot product performs the necessary sum over all the words (j-dimension)
def g_ham( x ) :
    return float(log_p0j.dot(x) + log_p0j_inv.dot((1-x)) + np.log(P_ham))

def g_spam( x ) :
    return float(log_p1j.dot(x) + log_p1j_inv.dot((1-x)) + np.log(P_spam))


Spams:  602 Hams:  3857


C:\Users\helge\AppData\Local\Temp\ipykernel_7960\154588584.py:22: RuntimeWarning: divide by zero encountered in log
  log_p0j = np.log(rel_freq_hams)
C:\Users\helge\AppData\Local\Temp\ipykernel_7960\154588584.py:23: RuntimeWarning: divide by zero encountered in log
  log_p1j = np.log(rel_freq_spams)


And now we are ready to test.

In [88]:
real_class = test_set[:, 0]

true_positives = 0
false_positives = 0
true_negatives = 0
false_negatives = 0
for i in range(N_test):
    msg = test_set[i,1:]
    y = real_class[i]
    msg = msg.reshape(len(msg),1)
    # print("Ham value:", g_ham(msg), "Spam value:", g_spam(msg), "Ground truth:",y==1,"Flagged:", g_spam(msg) > g_ham(msg) )
    if g_spam(msg) > g_ham(msg) :  # We predicted spam
        if y == 1:
            true_positives += 1
        else :
            false_positives += 1
    else :   # We predicted ham
        if y == 0:
            true_negatives += 1
        else :
            false_negatives += 1

# Confusion matrix
print("|",true_negatives, false_positives, "|")
print("|",false_negatives, true_positives, "|")
print("Accuracy:",(true_positives + true_negatives)/N_test)


| 966 4 |
| 16 129 |
Accuracy: 0.9820627802690582
